In [153]:
import re
import numpy as np
import pandas as pd
import pickle
import chromadb
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.schema.runnable import RunnableMap
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema.document import Document
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import EnsembleRetriever
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureOpenAIEmbeddings, AzureChatOpenAI, AzureOpenAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.llms.vllm import VLLMOpenAI
from langchain_community.llms import Ollama

import urllib.request
import json
from dotenv import load_dotenv
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

import getpass
import os
from operator import itemgetter
import lark
import urllib
import json

from langchain.schema.document import Document
from langchain_community.document_loaders import UnstructuredFileLoader, PyPDFLoader
from tempfile import NamedTemporaryFile
import pypdf
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tempfile
from langchain.chains import LLMChain

chunk_size = 1600 # Demie-page
chunk_overlap = 128

########################################
# Conf file with connexion credentials
import configparser
config = configparser.ConfigParser()
config.read('/Users/aba/Documents/dev/conf.ini')
mixtral_api_base = config['creds']['MIXTRAL_API_BASE']
mixtral_api_key = config['creds']['MIXTRAL_API_KEY']
gpt_api_base = "https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
gpt_api_key = config['creds']['GPT_API_KEY']

os.environ["MIXTRAL_API_BASE"] = config['creds']['MIXTRAL_API_BASE']
os.environ["MIXTRAL_API_KEY"] = config['creds']['MIXTRAL_API_KEY']
os.environ["GPT_API_KEY"] = config['creds']['GPT_API_KEY']
os.environ["AZURE_OPENAI_API_KEY"] = config['creds']['GPT_API_KEY']
os.environ["OPENAI_API_KEY"] = config['creds']['OPEN_AI_KEY_PERSO']
OPENAI_API_URL ="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
#OPENAI_API_URL ="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01"
OPENAI_API_URL="https://apigatewayinnovation.azure-api.net/openai-api/deployments/gpt-4o/chat/completions?api-version=2024-02-01"

from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


In [154]:
import os
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain.tools import StructuredTool
import requests
import json
import configparser
config = configparser.ConfigParser()
config.read('/Users/aba/Documents/dev/conf.ini')
# set up API key
os.environ["TAVILY_API_KEY"] = config['creds']['TAVILY_API_KEY'] 
from model import get_agent

In [158]:
key_word = "Comment faire une signature électronique"

In [155]:
def call_openai(prompt):
    try:
        url = OPENAI_API_URL

        hdr = {
            # Request headers
            'Content-Type': 'application/json',
            'Cache-Control': 'no-cache',
            'Ocp-Apim-Subscription-Key': os.environ['GPT_API_KEY'],
        }
        data = {
            "messages": [
                {
                    "role": "system",
                    "content": ""
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stop": ["\\n"],
            #"max_tokens" : 10000
        }
        #print(prompt)

        # Request body
        data = json.dumps(data)
        #print(data)
        
        req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))

        req.get_method = lambda: 'POST'
        response = urllib.request.urlopen(req)
        #print(response.status==200)
        return json.loads(response.read())["choices"][0]["message"]['content']
    except Exception as e:
        print(e)
        return None


class CustomGPT(LLM):
    """
    New definition of the LLM for interacting with LangChain

    """

    # n: int
    # """ parameter on initializing"""

    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> str:
        """Run the LLM on the given input.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of the stop substrings.
                If stop tokens are not supported consider raising NotImplementedError.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            The model output as a string. Actual completions SHOULD NOT include the prompt.
        """
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        return call_openai(prompt) #"Prompt : " + prompt + "\n Anwser : " +  call_openai(prompt)

    def _stream(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Stream the LLM on the given prompt.

        This method should be overridden by subclasses that support streaming.

        If not implemented, the default behavior of calls to stream will be to
        fallback to the non-streaming version of the model and return
        the output as a single chunk.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of these substrings.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            An iterator of GenerationChunks.
        """
        for char in call_openai(prompt):
            chunk = GenerationChunk(text=char)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)

            yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "GPTModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "customGPT"

    
def instantiate_mixtral() -> VLLMOpenAI:
    """Instantiates the Mixtral LLM."""

    llm = VLLMOpenAI(
        openai_api_key=os.environ["MIXTRAL_API_KEY"],
        openai_api_base=os.environ["MIXTRAL_API_BASE"],
        model_name="mistralai/Mixtral-8X7B-Instruct-v0.1",
        temperature=0.1,
        max_tokens=5000,
        verbose=True,
    )
    return llm



{'status': 'success',
 'datas': {'role': 'assistant',
  'result': '{\n  "Certificat numérique": ["clé privée", "clé publique", "chiffrement", "authentification", "sécurité", "identité numérique", "validité", "autorité de certification", "cryptographie", "algorithmes"],\n  "Règlementation": ["eIDAS", "normes", "directive européenne", "protection des données", "confidentialité", "loi", "obligations légales", "juridique", "intégrité", "responsabilités"],\n  "Logiciels de signature": ["Adobe Sign", "DocuSign", "SignRequest", "HelloSign", "eversign", "signature électronique avancée", "service en ligne", "plateforme dédiée", "authentification forte", "fonctionnalités"],\n  "Processus de validation": ["authentification multi-facteurs", "identification", "vérification", "documentation légale", "timestamping", "mentions obligatoires", "consentement éclairé", "traçabilité", "audit", "contrôle"],\n  "Avantages de la signature électronique": ["gain de temps", "efficacité", "réduction des coûts", "

In [175]:
## Trouver mots clés complémentaires
def get_complement_keywords(keyword):
    
    list_complement = []
    try:
        r = requests.post(url="https://semrank.io/admin/api/complement", data={"search": keyword, "spec": "words"})
        l_kw = json.loads(r.content)
        list_complement = sum([i for i in json.loads(l_kw["datas"]["result"]).values()], []) 
    except:
        pass
    
    
    return list_complement

complement = get_complement_keywords(key_word)

In [176]:
complement

['certificat numérique',
 'clé publique',
 'clé privée',
 'authentification',
 'cryptographie',
 'sécurité',
 'empreinte numérique',
 'protocole de chiffrement',
 'authentification forte',
 'certification électronique',
 'plateforme de signature électronique',
 'solution de gestion de signature',
 'outil de signature en ligne',
 'application de signature électronique',
 'logiciel de validation électronique',
 'gestionnaire de certificats',
 'logiciel de cryptographie',
 'application de sécurisation des documents',
 'application de sécurité numérique',
 'programme de chiffrement',
 'normes de signature électronique',
 'cadre légal des signatures électroniques',
 'réglementation numérique',
 'loi sur la signature électronique',
 'certification électronique légale',
 'directive européenne eIDAS',
 'acte écrit numérique',
 'validité juridique des signatures électroniques',
 'documentation électronique',
 'preuve électronique',
 'créer une signature électronique',
 'vérifier une signature n

0

In [4]:
# set up the agent
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)


def get_semrank_result(keyword: str) -> dict:
    """ Search for Semrank result on specific keywrods and parse it"""
    r = requests.post(url="https://semrank.io/admin/api/keywords", data={"query":keyword})
    return json.loads(r.content)


def get_hn_structures(keyword: str) -> dict:
    """Retrieves Hn structures from top 3 website of Semrank's results about specific keyword"""
    results = get_semrank_result(keyword)
    a = {}
    for _, concurrent in results["datas"]["concurrents"].items():
        if concurrent["position"]<=3:
            a[concurrent["url"]] = concurrent["headings"]
            a["title"] = concurrent["title"]
            a["descr"] = concurrent["descr"]
    
    return {"result" : results, "parsed": a}


tool_hn= StructuredTool.from_function(get_hn_structures)


tools = [tavily_tool, tool_hn]

def parse_semrank_object(results, top_concurrent = 3):
    
    # 1. related questions
    related_questions = [i["question"] for _,i in results["datas"]["paa"].items()] 
    other_related_questions = [i["query"] for _,i in results["datas"]['related'].items()]
    
    # 2. Concurrent + docapposte data content => Title, snippet, Content, Hn structure
    list_concurr = results["datas"]['concurrents']
    concurrents_data = []
    positions = sorted([c["position"] for _, c in results["datas"]["concurrents"].items()])
    docaposte_data = []
    all_url = []
    
    ## Data on "concurrents" key
    for _, concurrent in results["datas"]["concurrents"].items():
        a = {}
        all_url.append(concurrent["url"])
        a["position"] = concurrent["position"]
        a["title"] = concurrent["title"]
        a["snippet"] = concurrent["snippet"]
        a["url"] = concurrent["url"]
        a["descr"] = concurrent["descr"]
        a["headings"] = concurrent["headings"]
        a["nb_words"] = concurrent["nb_words"]
        a["content"] = concurrent["content"]
        ## TODO : vérifier si le site est mieux classé que docaposte
        if (concurrent["position"] in positions[:top_concurrent+1] and 
            len(concurrents_data) < top_concurrent and 
            "docaposte" not in concurrent["url"]) :
            concurrents_data.append(a)
        if "docaposte" in concurrent["url"]:
            docaposte_data.append(a)
    
    ## Data on 'backlinks' key
    for _, b in results["datas"]["backlinks"].items():
        a = {}
        'title', 'headings', 'snippet', 'url', 'position'
        a["position"] = b["position"]
        a["title"] = b["title"]
        a["snippet"] = b["snippet"]
        a["url"] = b["url"]
        a["headings"] = b["headings"]
        a["nb_words"] = 0
        a["content"] = ""
        if ("docaposte" in b["url"]) :
            docaposte_data.append(a)
    
    
    obj = {'related_questions' : related_questions, 'other_related_questions' : other_related_questions, 
           'docaposte_data' : docaposte_data, 'concurrents_data' : concurrents_data, 'all_url' : all_url}
    return obj  




In [5]:

target_url = "https://www.docaposte.com/blog/article/comment-creer-signature-electronique-document"

In [9]:
%%time
results = get_semrank_result(key_word)

CPU times: user 57.7 ms, sys: 6.42 ms, total: 64.1 ms
Wall time: 10.4 s


In [10]:
import requests
from bs4 import BeautifulSoup

def get_hn_structure_and_content(url):
    response = requests.get(url)
    response.raise_for_status()  # Vérifie si la requête a réussi

    soup = BeautifulSoup(response.content, 'html.parser')
    # Extraction du titre de la page
    page_title = soup.title.string.strip() if soup.title else 'No title found'

    # Extraction de la description de la page
    meta_description = soup.find('meta', attrs={'name': 'description'})
    page_description = meta_description['content'].strip() if meta_description else 'No description found'

    # Extraction de la structure Hn et du contenu associé
    hn_structure = []
    current_section = None

    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'ul']):
        if tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            if current_section:
                hn_structure.append(current_section)
            current_section = (tag.name.upper(), tag.get_text(strip=True), "")
        elif current_section:
            current_section = (current_section[0], current_section[1], current_section[2] + "\n" + tag.get_text(strip=True))

    if current_section:
        hn_structure.append(current_section)

    return page_title, page_description, hn_structure


page_title, page_description, hn_structure = get_hn_structure_and_content(target_url)

#print("Structure Hn et contenu:")
#for tag, title, content in hn_structure:
#    print(f"{tag}: {title}\nContenu:\n{content}\n")


In [11]:
parsed_res = parse_semrank_object(results, top_concurrent = 3)

In [12]:
keywords_list = [s.split(':')[0] for s in list(results['datas']['keywords_list'].values()) ]
#keywords_list

In [13]:
results

{'status': 'success',
 'datas': {'len_serper': 3,
  'len_contents': 10,
  'len_backlinks': 16,
  'len_backlinks_content': 9,
  'keywords_list': {'0': 'signature:32.0',
   '1': 'électronique:14.0',
   '2': 'document:9.3',
   '3': 'documents:6.9',
   '4': 'signer:6.8',
   '5': 'sélectionnez:5.7',
   '6': 'image:5.6',
   '7': 'signatures:5.4',
   '8': 'pdf:4.0',
   '9': 'nom:4.6',
   '10': 'mail:4.3',
   '11': 'cliquez:4.1',
   '12': 'processus:3.7',
   '13': 'ajouter:3.7',
   '14': 'niveau:3.6',
   '15': 'ligne:3.4',
   '16': 'signé:3.3',
   '17': 'créer:3.3',
   '18': 'simple:3.2',
   '19': 'signataire:3.2',
   '20': 'électroniques:3.2',
   '21': 'informations:3.2',
   '22': 'étape:3.2',
   '23': 'format:3.1',
   '24': 'sécurité:2.0',
   '25': 'solution:2.0',
   '26': 'certificat:2.0',
   '27': 'insérer:2.0',
   '28': 'inclure:2.9',
   '29': 'téléphone:2.8',
   '30': 'lien:2.8',
   '31': 'site:2.7',
   '32': 'identité:2.7',
   '33': 'confiance:2.6',
   '34': 'entreprises:2.6',
   '35': 

In [14]:
parsed_res

{'related_questions': ['Comment effectuer une signature électronique sur un document ?',
  'Comment me faire une signature électronique ?',
  'Comment signer électroniquement un document PDF ?',
  'Comment se créer une signature électronique gratuite ?'],
 'other_related_questions': ['Comment faire une signature électronique Word',
  'Signature électronique gratuite',
  'Comment faire une signature électronique sur un document PDF',
  'Comment faire une signature électronique par mail',
  'Comment faire une signature électronique gratuite',
  'Comment faire une signature électronique Gmail',
  'Comment faire une signature électronique Outlook',
  'Signature électronique exemple'],
 'docaposte_data': [{'position': 37,
   'title': 'Signature qualifiée ou avancée reposant sur un certificat qualifié ?',
   'snippet': "La signature électronique qualifiée (QES) est le moyen le plus complet et sécurisé pour attester de l'identité du signataire d'un document. Elle offre une ...",
   'url': 'ht

In [15]:
len(parsed_res["concurrents_data"])

3

In [16]:
################

### Embedding

In [17]:
## GPT4all
#gpt4all_embedding = GPT4AllEmbeddings()
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
gpt4all_embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

### LLM

In [18]:
# Mixtral
mistral_llm = instantiate_mixtral()
# OpenAI - GPT
gpt_llm = CustomGPT()

In [19]:
agent = get_agent(tools)
key_word = "Comment faire une signature électronique"
#res = agent({"input": key_word})

/Users/aba/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### Get & Parse data

In [20]:
%%time
results = get_semrank_result(key_word)

CPU times: user 55.1 ms, sys: 5.22 ms, total: 60.4 ms
Wall time: 10.3 s


In [21]:
def parse_semrank_object(results, top_concurrent = 3):
    
    # 1. related questions
    related_questions = [i["question"] for _,i in results["datas"]["paa"].items()] 
    other_related_questions = [i["query"] for _,i in results["datas"]['related'].items()]
    
    # 2. Concurrent + docapposte data content => Title, snippet, Content, Hn structure
    list_concurr = results["datas"]['concurrents']
    concurrents_data = []
    positions = sorted([c["position"] for _, c in results["datas"]["concurrents"].items()])
    docaposte_data = []
    all_url = []
    
    ## Data on "concurrents" key
    for _, concurrent in results["datas"]["concurrents"].items():
        a = {}
        all_url.append(concurrent["url"])
        a["position"] = concurrent["position"]
        a["title"] = concurrent["title"]
        a["snippet"] = concurrent["snippet"]
        a["url"] = concurrent["url"]
        a["descr"] = concurrent["descr"]
        a["headings"] = concurrent["headings"]
        a["nb_words"] = concurrent["nb_words"]
        a["content"] = concurrent["content"]
        if (concurrent["position"] in positions[:top_concurrent+1] and 
            len(concurrents_data) < top_concurrent and 
            "docaposte" not in concurrent["url"]) :
            concurrents_data.append(a)
        if "docaposte" in concurrent["url"]:
            docaposte_data.append(a)
    
    ## Data on 'backlinks' key
    for _, b in results["datas"]["backlinks"].items():
        a = {}
        'title', 'headings', 'snippet', 'url', 'position'
        a["position"] = b["position"]
        a["title"] = b["title"]
        a["snippet"] = b["snippet"]
        a["url"] = b["url"]
        a["headings"] = b["headings"]
        a["nb_words"] = 0
        a["content"] = ""
        if ("docaposte" not in b["url"]) :
            concurrents_data.append(a)
        else:
            docaposte_data.append(a)
    
    
    obj = {'related_questions' : related_questions, 'other_related_questions' : other_related_questions, 
           'docaposte_data' : docaposte_data, 'concurrents_data' : concurrents_data, 'all_url' : all_url}
    return obj  

In [22]:
def parse_hn_from_dict(obj):
    
    # res = ['<' + h['type'] + '>' + h['text'] + '<' + h['type'] + '/>'  for _, h in obj.items() ]
    res = [ (h['type'], h['text'])  for _, h in obj.items() ]
    
    return res #'  \n'.join(res)


In [23]:
parsed_res = parse_semrank_object(results, top_concurrent = 3)

In [24]:
parse_hn_from_dict(parsed_res["concurrents_data"][0]["headings"])

[('h1', 'Comment faire une signature électronique'),
 ('h2', 'La signature électronique Comment ça marche'),
 ('h3', 'Fonctionnement'),
 ('h2', 'Valeur probante et validité'),
 ('h3', 'Identification multi-facteurs'),
 ('h3', 'Intégrité garantie'),
 ('h3', 'Dossier de preuves horodaté'),
 ('h2', 'Comment faire une signature électronique sur mes documents'),
 ('h3', 'De la consultation à l’approbation'),
 ('h3', 'Consultation de la demande'),
 ('h3', 'Validation de votre identité'),
 ('h3', 'Accord et signature'),
 ('h2', 'Faire une signature électronique avec l’application Yousign'),
 ('h3', 'Créer un compte sur l’application Yousign'),
 ('h3', 'Ajouter des contrats'),
 ('h3', 'Choisir des signataires'),
 ('h3', 'Notifications et signatures'),
 ('h2', 'Comment faire une signature numérique depuis mes outils'),
 ('h3', 'Une expérience fluide et sans friction'),
 ('h3', 'Une intégration dans vos outils'),
 ('h3', 'La simplicité pour votre équipe'),
 ('h3', 'Intégrez la signature électron

#### Proposition structure Hn

In [25]:
target_hn_structure = [(tag.lower(), title) for tag, title, content in hn_structure]
str(target_hn_structure)

'[(\'h1\', \'Comment faire une signature électronique sur un document ?\'), (\'h2\', "L\'essentiel à retenir sur la création d\'une signature électronique sur un document"), (\'h2\', \'Qu’est-ce qu’une signature électronique ?\'), (\'h2\', \'Quels types de documents peut-on signer électroniquement ?\'), (\'h2\', \'Quels sont les avantages de la signature électronique ?\'), (\'h2\', "Les différentes étapes pour la création d\'une signature électronique"), (\'h2\', \'Les différents moyens de créer sa signature pour signer électroniquement un document\'), (\'h2\', \'La gamme de signature électronique de Docaposte\'), (\'h2\', \'Dans l’actualité\')]'

In [26]:
n_concurrents = len(parsed_res["concurrents_data"])
concurrents_hn_structure =  [parse_hn_from_dict(parsed_res["concurrents_data"][i]["headings"]) for i in range(n_concurrents) ]
concurrents_hn_structure =  [[list(j) for j in i] for i in concurrents_hn_structure ]

In [27]:
related_questions = parsed_res["related_questions"] + parsed_res["other_related_questions"]
prompt_structure = """
Vous êtes l'assistant d'un consultant SEO qui doit proposer un contenu pour le site web de son entreprise. Pour cela,
vous disposez de plusieurs exemples de structure Hn de pages bien référencées, parlant du sujet. Ces exemples doivent
vous servir pour proposer 3 nouvelles structures Hn pour aider le consultant SEO à rédiger son contenu. Vous utiliserez
aussi les questions que se posent souvent les internautes sur ce sujet pour adapter votre proposition de structure Hn.
Votre réponse doit être uniquement un dictionnaire de 3 listes de listes Python, qui donnent les 3 structures Hn à
utiliser pour le contenu. Votre réponse doit être comme dans les exemples ci-dessous, sans aucun commentaire.

### Entrées :

1. **Sujet de l'article à rédiger** :
{keyword}

2. **Autres questions similaires que se posent les internautes sur le sujet** :
{related_questions}

3. **Exemples structures Hn des pages les mieux référencées** :
{exemples_structures}

### Sortie attendue :
Un dictionnaire contenant trois propositions de structure Hn. Chaque proposition doit être sous forme de liste ayant des sous listes composés de 2 éléments sous ce format : ["h1", "le titre"].

Exemple de sortie :
{
    "proposition1" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]],
    "proposition2" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]],
    "proposition3" : [["h1", "Mon titre 1"], ["h2", "Mon sous-titre 1"], ["h2", "Mon sous-titre 2"], ["h1", "Mon titre 2"], ["h1", "Mon titre 3"]]
}

### Instructions :

1. Respecter strictement les formats demandés.
2. S'inspirer des exemples de structure fournis.
3. Utiliser des guillemets doubles pour tout texte dans le résultat.
""".replace("{exemples_structures}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_hn_structure ]))\
.replace("{related_questions}", str("\n".join(related_questions)))\
.replace("{keyword}", str(key_word))


### Intention de recherche

In [28]:
%%time
prompt_intention = f"""Quelle est l'intention de recherche d'une personne tapant dans Google le mot-clé : "{key_word}"? Que veut-elle savoir, 
retrouver dans les contenus qui vont apparaitre ?"""

intention = gpt_llm.invoke(prompt_intention)

CPU times: user 58.3 ms, sys: 6 ms, total: 64.3 ms
Wall time: 4.4 s


In [29]:
intention

'Lorsqu\'une personne tape le mot-clé "Comment faire une signature électronique" dans Google, son intention de recherche est probablement de comprendre les étapes et les outils nécessaires pour créer une signature électronique valide. Voici ce qu\'elle souhaite généralement savoir ou retrouver dans les contenus qui apparaîtront :\n\n1. **Définition et Explications**: Qu\'est-ce qu\'une signature électronique et comment fonctionne-t-elle?\n2. **Étapes Pratiques**: Un guide étape par étape sur la façon de créer une signature électronique.\n3. **Outils et Logiciels**: Des recommandations d\'outils ou de logiciels populaires et fiables pour créer une signature électronique.\n4. **Tutoriels Vidéo et Illustrations**: Des vidéos ou des guides illustrés pour faciliter la compréhension du processus.\n5. **Légalité et Sécurité**: Des informations sur la légalité, la sécurité et la validité des signatures électroniques dans différents contextes ou juridictions.\n6. **Applications**: Des exemples 

In [30]:
%%time
prompt_init_structure = f"""Nous devons rédiger un article de blog sur la thématique "{key_word}". Tu as ci-dessous un descriptif de l'intention de recherche d'une personne 
sur ce sujet : ce qu'il aimerait savoir. Sur la base de ces informations, quelle structure HN aurais-tu proposé  pour un article de blog ? Donne la structure Hn en markdown.
## Intention de recherche : 
"{intention}"
Structure Hn : 
"""
init_structure = gpt_llm.invoke(prompt_init_structure)

CPU times: user 53.5 ms, sys: 4.13 ms, total: 57.7 ms
Wall time: 18.3 s


In [31]:
init_structure

'Voici une structure Hn en Markdown pour un article de blog sur "Comment faire une signature électronique" :\n\n```markdown\n# Comment faire une signature électronique\n\n## Qu\'est-ce qu\'une signature électronique ?\n\n### Définition\n- Explication de ce qu\'est une signature électronique\n- Différence entre une signature électronique et une signature manuscrite\n\n### Fonctionnement\n- Comment fonctionne une signature électronique\n- Les principes technologiques derrière les signatures électroniques\n\n## Guide étape par étape pour créer une signature électronique\n\n### Étape 1: Choisir un outil ou un logiciel\n- Introduction aux différents types d\'outils disponibles\n- Critères pour choisir le bon outil\n\n### Étape 2: Créer et configurer votre compte\n- Procédure pour s\'inscrire sur une plateforme de signature électronique\n\n### Étape 3: Importer les documents\n- Comment importer ou télécharger les documents à signer\n\n### Étape 4: Signer électroniquement\n- Explication des d

In [32]:
def md_to_list_hn(markdown_text):
    # Utiliser une expression régulière pour identifier les titres Hn
    hn_pattern = re.compile(r'^(#+)\s*(.+)')
    
    # Initialiser une liste pour stocker la structure Hn
    structure = []
    
    # Parcourir chaque ligne du texte markdown
    for line in markdown_text.splitlines():
        match = hn_pattern.match(line)
        if match:
            # Obtenir le niveau de l'en-tête (nombre de '#')
            level = len(match.group(1))
            # Obtenir le titre de l'en-tête
            title = match.group(2).strip()
            # Ajouter à la structure en utilisant le format ["hN", "titre"]
            structure.append([f"h{level}", title])
    
    return structure

md_to_list_hn(init_structure)

[['h1', 'Comment faire une signature électronique'],
 ['h2', "Qu'est-ce qu'une signature électronique ?"],
 ['h3', 'Définition'],
 ['h3', 'Fonctionnement'],
 ['h2', 'Guide étape par étape pour créer une signature électronique'],
 ['h3', 'Étape 1: Choisir un outil ou un logiciel'],
 ['h3', 'Étape 2: Créer et configurer votre compte'],
 ['h3', 'Étape 3: Importer les documents'],
 ['h3', 'Étape 4: Signer électroniquement'],
 ['h3', 'Étape 5: Enregistrement et vérification de la signature'],
 ['h2', 'Outils et logiciels pour créer une signature électronique'],
 ['h3', 'Comparaison des principaux outils'],
 ['h3', 'Caractéristiques clés à rechercher'],
 ['h2', 'Tutoriels vidéo et guides illustrés'],
 ['h3', 'Vidéo tutoriels'],
 ['h3', 'Guides illustrés'],
 ['h2', 'Légalité et sécurité des signatures électroniques'],
 ['h3', 'Légalité'],
 ['h3', 'Sécurité'],
 ['h2', 'Applications des signatures électroniques'],
 ['h3', 'Situations courantes'],
 ['h3', 'Types de documents'],
 ['h2',
  "Trucs 

In [33]:
## Multiple suggestions

In [34]:
%%time
related_questions = parsed_res["related_questions"] + parsed_res["other_related_questions"]
prompt_structure = """
Vous êtes un employé SEO de Docaposte. Vous devez écrire un article de blog pour le site web de Docaposte, sur un sujet donné. Pour cela,
vous disposez d'un premier brouillon de structure Hn que vous devez améliorer en se basant sur les éléments suivants : 
- Plusieurs exemples de structure Hn de pages bien référencées, parlant du sujet. 
- Des questions fréquentes que se posent les internautes sur le sujet.
Ces informations doivent vous permettre de proposer 3 version améliorées de structure Hn pour l'article de blog à rédiger. Les 3 propositions
doivent se différencier sur certains points thématiques abordés.
Votre réponse doit être uniquement un dictionnaire de 3 listes de listes Python, qui donnent les 3 structures Hn à
utiliser pour le contenu. Votre réponse doit être comme dans les exemples ci-dessous, sans aucun commentaire.

### Entrées :

1. **Sujet de l'article à rédiger** :
{keyword}

2. **Premier brouillon de structure Hn** :
{init_structure}

3. **Exemples structures Hn des pages les mieux référencées** :
{exemples_structures}

4. **Autres questions fréquentes que se posent les internautes sur le sujet** :
{related_questions}


### Sortie attendue :
Un dictionnaire contenant trois propositions de structure Hn. Chaque proposition doit être sous forme de liste ayant 
des sous listes composés de 2 éléments sous ce format : ["h1", "le titre"].

Exemple de sortie :
    {{
        "proposition1": [
                                ["h1", "Mon titre 1"], 
                                ["h2", "Mon sous-titre 1"], 
                                ["h2", "Mon sous-titre 2"], 
                                ["h1", "Mon titre 2"], 
                                ["h1", "Mon titre 3"]
        ],
        "proposition2": [
                                ["h1", "Mon titre 1"], 
                                ["h2", "Mon sous-titre 1"], 
                                ["h2", "Mon sous-titre 2"], 
                                ["h1", "Mon titre 2"], 
                                ["h1", "Mon titre 3"]
        ],
        "proposition3": [
                                ["h1", "Mon titre 1"], 
                                ["h2", "Mon sous-titre 1"], 
                                ["h2", "Mon sous-titre 2"], 
                                ["h1", "Mon titre 2"], 
                                ["h1", "Mon titre 3"]
        ]
    }}
    
    ### Instructions :
    
    1. S'inspirer des exemples de structure fournis.
    2. Utiliser des guillemets doubles pour tout texte dans le résultat.
    3. Ne pas proposer un seul sous-niveau de titre Hn inférieur en dessous d'un niveau H(n+1) : par exemple, ne pas 
    proposer un seul H3 sous un H2.
    4. Ne proposer qu'un seul niveau H1 qui présente le sujet de l'article de blog. Sa formulation doit être captivante, adaptée pour un article de blog.
    5. Au moins une des autres questions fréquentes que se posent les internautes sur le sujet doit être pris en 
    compte dans la structure proposée.
    6. Dans la formulation des phrase, utilisez un ton adapté pour des articles de blog
""".replace("{exemples_structures}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_hn_structure ]))\
.replace("{related_questions}", str("\n".join(related_questions)))\
.replace("{keyword}", str(key_word))\
.replace("{init_structure}", str(init_structure))

propositions_structure = gpt_llm.invoke(prompt_structure)

CPU times: user 55.9 ms, sys: 5.01 ms, total: 60.9 ms
Wall time: 15.6 s


In [35]:
propositions_structure

'```python\n{\n    "proposition1": [\n        ["h1", "Découvrez Comment Faire une Signature Électronique Facilement"],\n        ["h2", "Qu\'est-ce qu\'une signature électronique ?"],\n        ["h3", "Définition"],\n        ["h3", "Différence avec la signature manuscrite"],\n        ["h2", "Pourquoi utiliser une signature électronique ?"],\n        ["h3", "Avantages"],\n        ["h3", "Applications pratiques"],\n        ["h2", "Guide étape par étape pour créer une signature électronique"],\n        ["h3", "Étape 1 : Choisir un outil ou un logiciel"],\n        ["h3", "Étape 2: Créer et configurer votre compte"],\n        ["h3", "Étape 3: Importer les documents"],\n        ["h3", "Étape 4: Signer électroniquement"],\n        ["h3", "Étape 5: Enregistrer et vérifier la signature"],\n        ["h2", "Comment faire une signature électronique sur différents formats ?"],\n        ["h3", "Sur un document PDF"],\n        ["h3", "Sur un document Word"],\n        ["h2", "Outils et logiciels recomma

In [36]:
%%time
#Intention de recherche
def get_intention_recherche(keyword, llm):
    question = f"""Quels sont les principales informations clés que cherche à savoir l'internaute à travers la question suivante : \n {keyword}. 
    Donne une réponse synthétique avec les différents points."""
    intention: str = llm.invoke(question)

    return intention

get_intention_recherche(key_word, gpt_llm)

CPU times: user 53.4 ms, sys: 3.63 ms, total: 57 ms
Wall time: 3.06 s


'Lorsqu\'un internaute pose la question "Comment faire une signature électronique ?", il cherche généralement à obtenir les informations clés suivantes :\n\n1. **Définition et Utilité** : Ce qu\'est une signature électronique et les situations où elle est nécessaire.\n2. **Outils et Logiciels** : Les plateformes et logiciels disponibles pour créer une signature électronique (par exemple, Adobe Sign, DocuSign, etc.).\n3. **Étapes Générales** : Les étapes de base pour créer une signature électronique, typiquement :\n    - Sélection de l’outil/logiciel\n    - Téléchargement du document\n    - Création et application de la signature\n    - Sauvegarde et envoi du document signé\n4. **Légalité et Conformité** : Les aspects juridiques associés à l\'utilisation des signatures électroniques et comment s\'assurer qu\'elles sont conformes aux réglementations locales ou internationales.\n5. **Sécurité**: Les mesures de sécurité pour garantir l\'authenticité et l\'intégrité des documents signés éle

In [37]:
#prompt_structure
class StructureHn(BaseModel):
    structure: dict[str, list[list[str, str]]] = Field(description="Dictionnaire de 3 propositions de structure Hn. Ceux-ci sont au format de liste avec 2 éléments : le niveau (h1,h2, etc) et le titre")

class Content(BaseModel):
    content: dict[str, list[list[str, str, str]]] = Field(description="Dictionnaire de 3 propositions de contenu pour une structure Hn. Chaque proposition est au format de liste avec 3 éléments : le niveau (h1,h2, etc), le titre et le contenu")



def build_chain(llm, output_class):
    parser = PydanticOutputParser(pydantic_object=output_class)

    prompt = PromptTemplate(
        template="Answer the user query based on the information given.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser
    
    return chain


chain_structure = build_chain(gpt_llm, StructureHn)
chain_content = build_chain(gpt_llm, Content)

In [38]:
%%time
res_struct_gpt = chain_structure.invoke({"query": prompt_structure})
res_struct_gpt

CPU times: user 93.7 ms, sys: 4.65 ms, total: 98.4 ms
Wall time: 10.9 s


StructureHn(structure={'proposition1': [['h1', 'Comment faire une signature électronique : Guide complet'], ['h2', "Qu'est-ce qu'une signature électronique ?"], ['h3', 'Définition et avantages'], ['h3', 'Différence avec la signature manuscrite'], ['h2', 'Comment fonctionne une signature électronique ?'], ['h3', 'Les principes technologiques'], ['h3', 'Les étapes de vérification'], ['h2', 'Étapes pour créer une signature électronique'], ['h3', 'Choisir le bon outil ou logiciel'], ['h3', 'Configurer votre compte'], ['h3', 'Importer des documents'], ['h3', 'Signer électroniquement'], ['h3', 'Sauvegarder et vérifier la signature'], ['h2', 'Les meilleurs outils pour créer une signature électronique'], ['h3', 'Comparaison des outils populaires'], ['h3', 'Caractéristiques clés à rechercher'], ['h2', 'Légalité et sécurité des signatures électroniques'], ['h3', 'Réglementations par pays'], ['h3', 'Mesures de sécurité'], ['h2', 'FAQ : Questions fréquentes'], ['h3', 'Comment signer un document PD

In [39]:
res_struct_gpt.structure

{'proposition1': [['h1',
   'Comment faire une signature électronique : Guide complet'],
  ['h2', "Qu'est-ce qu'une signature électronique ?"],
  ['h3', 'Définition et avantages'],
  ['h3', 'Différence avec la signature manuscrite'],
  ['h2', 'Comment fonctionne une signature électronique ?'],
  ['h3', 'Les principes technologiques'],
  ['h3', 'Les étapes de vérification'],
  ['h2', 'Étapes pour créer une signature électronique'],
  ['h3', 'Choisir le bon outil ou logiciel'],
  ['h3', 'Configurer votre compte'],
  ['h3', 'Importer des documents'],
  ['h3', 'Signer électroniquement'],
  ['h3', 'Sauvegarder et vérifier la signature'],
  ['h2', 'Les meilleurs outils pour créer une signature électronique'],
  ['h3', 'Comparaison des outils populaires'],
  ['h3', 'Caractéristiques clés à rechercher'],
  ['h2', 'Légalité et sécurité des signatures électroniques'],
  ['h3', 'Réglementations par pays'],
  ['h3', 'Mesures de sécurité'],
  ['h2', 'FAQ : Questions fréquentes'],
  ['h3', 'Comment 

In [139]:
def generate_prompt_part_content(subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, article_start="", part_cible=""):
    prompt = f"""
    Vous êtes un employé SEO de Docaposte. Vous devez participer à la rédaction d'un article de blog pour le site 
    web de Docaposte, sur un sujet donné. Vous avez à votre charge la rédaction d'une seule section de l'article.
    Pour cela, vous disposez d'une structure HN que l'article doit respecter. Vous avez aussi 3 versions du début 
    de l'article déjà rédigé et vous devez fournir 3 propositions de contenu pour une section. 
    Tous les éléments en entrée sont les suivants :
        - le sujet de l'article de blog à rédiger;
        - l'intention de recherche des internautes qui se posent des questions sur ce sujet;
        - les autres questions fréquentes que se posent les internautes sur le sujet;
        - les mots clés important à faire figurer dans le contenu pour un meilleur référencement de l'article sur google;
        - la structure HN à respecter, sous forme d'une liste avec des sous-listes du type ["h1", "titre de la section"];
        - des exemples de contenus d'articles, parlant du sujet;
        - 3 versions pour le début de l'article déjà rédigé;
        - La partie de l'article à rédiger.
    Ces informations doivent vous permettre de fournir 3 propositions de contenu pour une section de l'article de blog à 
    rédiger, adaptées aux 3 versions du début de l'article déjà rédigées. Les 3 propositions doivent se différencier et 
    être des suites logiques aux 3 versions de début de l'article déjà rédigées.

    ### Entrées :

    1. **Sujet de l'article à rédiger** :
        {subject}

    2. **Intention de recherche des utilisateurs à travers ce sujet** :
        {intention_recherche}

    3. **Autres questions similaires que se posent les internautes sur le sujet** :
        {related_questions}

    4. **Mots clés importants pour l'article à rédiger** :
        {mots_cles}

    4. **Structure Hn à respecter** :
        {structure_hn}

    5. **Exemples d'articles** :
        {examples}
        
    6. **3 versions de début de l'article déjà rédigé** :
        {article_start}
        
    7. **Partie de l'article à rédiger** :
        {part_cible}

    ### Sortie attendue :

    Une liste contenant trois propositions de texte pour la partie correspondante de l'article, chaque proposition 
    étant un string et cohérent avec le début d'article fourni.

    Exemple de sortie :
    [
     "Contenu généré pour la proposition 1",
     "Contenu généré pour la proposition 2",
     "Contenu généré pour la proposition 3",
    ]

    ### Instructions :

    1. Respecter strictement le format de sortie avec une liste de 3 propositions de contenu.
    2. Dans la formulation des phrases, utiliser un ton adapté pour des articles de blog. S'adresser au
    lecteur en le vouvoyant et en parlant pour le compte de Docaposte. 
    3. Le ton doit être également rassurant, expert et pédagogique, qui vulgarise le contenu technique tout en ajoutant 
    des éléments de langage "expert" adaptés au sujet.
    4. Utiliser des guillemets doubles pour tout texte dans le résultat.
    5. Différencier les 3 propositions en donnant une qui propose plusieurs exemples avec pédagogie, une autre qui 
    contient des transitions originales entre les parties et une dernière qui soit très détaillée.
    6. Éviter de nommer des solutions ou entreprises concurrentes à Docaposte.
    7. Détailler le contenu avec des explications illustrées au besoin.
    8. Ne pas empiéter sur une autre partie de la structure HN. Se limiter à donner des informations spécifiques à la partie cible de l'article à rédiger.
    9. Il faut obligatoirement une liste de 3 propositions.
    10. Il faut nécessairement que les 3 propositions soient des suites logiques des 3 versions de début de l'article fournies.
    """
    return prompt


def generate_prompt_reformulate_content(subject, structure_hn, examples, len_content, intention_recherche, mots_cles,
                                        related_questions, list_ancre="", article_init=""):
    prompt = f"""
    Vous êtes un employé SEO de Docaposte. Vous devez écrire un article de blog pour le site web de Docaposte, sur un 
    sujet donné. Pour cela, vous disposez d'une structure HN que l'article doit respecter et d'un premier brouillon de 
    contenu d'article. Vous devez utiliser ces inputs pour proposer une nouvelle version reformulée et résumée de 
    l'article, cohérent, avec des transitions entre les différentes parties. Pour cela, vous disposez également des 
    éléments complémentaires suivants:
        - le sujet de l'article de blog à rédiger;
        - l'intention de recherche des internautes qui se posent des questions sur ce sujet;
        - les autres questions fréquentes que se posent les internautes sur le sujet;
        - les mots clés important à faire figurer dans le contenu pour un meilleur référencement de l'article sur google;
        - la structure HN à respecter, sous forme d'une liste avec des sous-listes du type ["h1", "titre de la section"];
        - des titres et liens d'articles similaires au sujet et disponible sur le site de Docaposte ;
        - des exemples de contenus d'articles, parlant du sujet.
        
    ### Entrées :

    1. **Sujet de l'article à rédiger** :
        {subject}

    2. **Intention de recherche des utilisateurs à travers ce sujet** :
        {intention_recherche}

    3. **Autres questions similaires que se posent les internautes sur le sujet** :
        {related_questions}

    4. **Mots clés importants pour l'article à rédiger** :
        {mots_cles}

    5. **Structure Hn à respecter** :
        {structure_hn}

    6. **Titre des thématiques similaires abordées dans des pages de Docaposte ainsi que les liens correspondants** :
        {list_ancre}

    7. **Exemples d'articles** :
        {examples}

    8. **Premiere version de l'article de blog** :
        {article_init}

    ### Sortie attendue :

    L'article de blog reformulé et respectant la structure HN fournie en entrée ainsi que le format de la première 
    version. Le texte doit être écrit en format markdown.

    ### Instructions :

    1. Reformuler le brouillon fourni en donnant un texte finale contenant environ {len_content} mots et en 
    respecter strictement le format de sortie demandé.
    2. Dans la formulation des phrases, utiliser un ton adapté pour des articles de blog. S'adresser au
    lecteur en le vouvoyant et en parlant pour le compte de Docaposte. 
    3. Mettre un paragraphe d'introduction à l'article de blog après le H1 qui correspond au titre. Les autres titres de 
    section également devront avoir un paragraphe introduction pour parler brièvement du contenu de la section.
    4. Le ton doit être également rassurant, expert et pédagogique, qui vulgarise le contenu technique tout en ajoutant 
    des éléments de langage "expert" adaptés au sujet.
    5. Utiliser des guillemets doubles pour tout texte dans le résultat.
    6. Vous devez ABSOLUMENT insérer dans certains passages du texte des liens vers les thématiques similaires abordées dans les pages de Docaposte, 
    données en entrée ci-haut. Pour cela, quand un mot ou groupe de mots présent dans le titre d'une des thématiques 
    similaires fournies apparait dans le brouillon d'article, dans la reformulation, mettre immédiatement après ce mot ou groupe 
    de mots le lien correspondant à cette thématique entre des parenthèses. Par exemple, si la thématique fournie similaire 
    est "Titre : 'signature électronique' | Lien : https://www.docaposte.com/solutions/signature-electronique", et 
    que le brouillon d'article contient "Pour sécuriser vos documents, une signature électronique est essentielle...",
    vous pourrez mettre dans le texte reformulé un passage comme suit : 
    "Pour sécuriser vos documents, une signature électronique (https://www.docaposte.com/solutions/signature-electronique) est essentielle...".
    L'insertion de la citation devra être naturelle. Il faut nécessairement que le
    lien inséré entre parenthèses soit après un mot ou groupe de mots présent dans le titre de la thématique similaire.
    Et il faudra également varier les thématiques citées.
    7. Éviter de nommer des solutions ou entreprises concurrentes à Docaposte.
    8. Détailler le contenu mais éliminer les répétitions.
    """
    return prompt

def extract_title_and_snippet(link):
    try:
        response = requests.get(link, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the title of the page
        title = soup.title.string if soup.title else "No title found"

        # Extract a paragraph or snippet as a summary
        paragraphs = soup.find_all('p')
        snippet = paragraphs[0].get_text() if paragraphs else "No snippet found"

        return title, snippet
    except Exception as e:
        return "No title found", "No snippet found"
    
def format_ancre(links):
    formatted_links = []
    for link in links:
        title, snippet = extract_title_and_snippet(link)
        # Get last part of the article
        last_part = link.rstrip('/').split('/')[-1]
        ancre = ' '.join(last_part.replace('-', ' ').replace('_', ' ').split())

        formatted_links.append(f"""Titre : '{ancre}' | Lien : {link}""")
    return "\n".join(formatted_links)

def google_search(query, site="docaposte.com", num_results=5):
    complete_query = f"{query} site:{site}"
    links = []
    for result in search(complete_query, num_results=num_results, lang="fr"):
        links.append(result)
    return links

In [116]:
from googlesearch import search
subject = key_word
intention_recherche = get_intention_recherche(key_word, gpt_llm)
related_questions = parsed_res["related_questions"]
mots_cles = keywords_list = [s.split(':')[0] for s in list(results['datas']['keywords_list'].values())]
structure_hn = res_struct_gpt.structure["proposition1"]
results_docaposte_content = google_search(key_word)
list_ancre = format_ancre(results_docaposte_content).replace("\n", "\n\n")
examples = [i['content'] for i in parsed_res["concurrents_data"]]
article_start = ": ".join((structure_hn[0]))

In [117]:
class PartContent(BaseModel):
    content: list[str, str, str] = Field(
        description="Liste de 3 propositions de contenu pour une partie de l'article de blog. Chaque proposition est un string")


chain_part_content = build_chain(gpt_llm, PartContent)

In [120]:
%%time
i = 1
part_cible = ": ".join((structure_hn[i])) 
print(part_cible)
prompt_part_content = generate_prompt_part_content(subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, article_start, part_cible)

res = chain_part_content.invoke({"query": prompt_part_content})
res.content

h2: Qu'est-ce qu'une signature électronique ?
CPU times: user 79.9 ms, sys: 13.5 ms, total: 93.3 ms
Wall time: 5.32 s


["La signature électronique est devenue un outil indispensable dans le monde numérique d'aujourd'hui. Mais qu'est-ce vraiment ? Une signature électronique est une méthode permettant de valider l'authenticité d'un document électronique de manière sécurisée. Contrairement à la signature manuscrite, elle utilise des algorithmes de cryptographie pour garantir que le document n'a pas été altéré après sa signature et que le signataire est bien celui qu'il prétend être. Par exemple, lorsque vous signez un document PDF électroniquement, vous pouvez utiliser votre nom écrit, une image de votre signature manuscrite, ou même tracer votre signature directement sur un écran tactile. Cette flexibilité et cette sécurité rendent la signature électronique particulièrement utile pour les entreprises et les particuliers qui cherchent à simplifier leurs processus administratifs tout en assurant une conformité légale stricte.",
 "En termes simples, une signature électronique est une donnée électronique qui

In [122]:
## TODO : faire une boucle while sur le contenu => ne pas s'arreter tant qu'on a pas 3 proposition, sinon compiler les propositions
def get_part_content_proposals(chain, subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, article_start, part_cible):
    
    proposals = []
    counter = 0
    while (counter < 3) and (len(proposals) < 3):
        prompt_part_content = generate_prompt_part_content(subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, article_start, part_cible)
        res = chain.invoke({"query": prompt_part_content})
        new_proposal = res.content
        if (len(new_proposal) == 3):
            proposals = new_proposal
        else:
            proposals += new_proposal
        counter += 1
    
    print(f"==> Did {counter} iterations to generate proposals")
    return(proposals)

get_part_content_proposals(chain_part_content, subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, article_start, part_cible)

==> Did 1 iterations to generate proposals


["Une signature électronique est une méthode moderne et sécurisée de valider des documents en ligne. Contrairement à la signature manuscrite, elle ne se contente pas de reproduire visuellement votre signature mais ajoute une couche de sécurité qui garantit l'authenticité et l’intégrité du signataire, ainsi que du document signé. Pour illustrer, imaginez-vous recevoir un contrat par e-mail : grâce à la signature électronique, vous pouvez rapidement, en quelques clics, signer ce contrat, et ce, depuis n'importe quel appareil connecté à internet. Le procédé est non seulement rapide, mais il assure également que le document n'a pas été modifié après la signature. Cette technologie est devenue indispensable tant pour les particuliers que pour les entreprises. Maintenant que nous avons défini ce qu'est une signature électronique, découvrons ses nombreux avantages dans la section suivante.",
 "La signature électronique, qu'est-ce que c'est exactement ? Il s'agit d'une méthode digitale permett

In [123]:
## Boucle sur les thématiques pour générer le texte en entier
def get_all_content_proposals(chain, subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions):
    
    
    articles = ": ".join((structure_hn[0]))
    articles = [articles, articles, articles]
    for i in range(len(structure_hn)):
        if i == 0:
            continue
        
        part_cible = ": ".join((structure_hn[i]))
        debut_art = "\n\n---\n".join(["Version " + str(a+1) + ": \n\n" + articles[a] for a in range(len(articles))])
        proposals = get_part_content_proposals(chain, subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions, debut_art, part_cible)
        print()
        articles = [art + "\n\n" + part_cible  + "\n\n" + prop for art, prop in zip(articles, proposals)]
        
        
    return articles
## combien de propositions initiales utiliser pour la nouvelle génération 

In [124]:
%%time
draft_content_proposals = get_all_content_proposals(chain_part_content, subject, structure_hn, examples, intention_recherche, mots_cles,
                                 related_questions)

==> Did 3 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 3 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 2 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 2 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 3 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 3 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 1 iterations to generate proposals

==> Did 2 iterations to generate

In [125]:
len(draft_content_proposals)

3

In [126]:
draft_content_proposals[0]

"h1: Comment faire une signature électronique : Guide complet\n\nh2: Qu'est-ce qu'une signature électronique ?\n\nUne signature électronique est un procédé permettant de signer des documents numériquement, en garantissant l'intégrité et l'authenticité du document signé. Mais qu'est-ce qui rend cette méthode à la fois sécurisée et fiable ? Contrairement à une signature manuscrite qui peut être facilement falsifiée, une signature électronique utilise des algorithmes cryptographiques pour assurer qu'aucune modification ne peut être opérée sur le document sans être détectée. Par exemple, une signature numérique peut inclure un certificat numérique issu d'une autorité de certification reconnue, certifiant l'identité du signataire et criant une liaison indissociable entre le signataire et le document électronique. En fin de compte, la signature électronique constitue un gage de sécurité et de conformité dans notre ère numérique, simplifiant et accélérant les processus sans compromettre la fi

In [140]:
## Reformulation
len_content = 2000
prompt_reformulate_content = [generate_prompt_reformulate_content(subject, structure_hn, examples, len_content, intention_recherche, mots_cles,
                                        related_questions, list_ancre, a) for a in draft_content_proposals]

In [141]:
%%time 
final_articles = [gpt_llm.invoke(p) for p in prompt_reformulate_content]

CPU times: user 143 ms, sys: 8.92 ms, total: 152 ms
Wall time: 1min 13s


In [144]:
final_articles[2]

"```markdown\n# Comment faire une signature électronique : Guide complet\n\nDans un monde en constante digitalisation, la signature électronique s'impose comme un outil majeur pour sécuriser et simplifier vos transactions. Ce guide complet vous fournira toutes les informations nécessaires et les étapes détaillées pour créer, utiliser et comprendre l'importance des signatures électroniques.\n\n## Qu'est-ce qu'une signature électronique ?\n\n### Définition et avantages\n\nUne signature électronique est une méthode de validation numérique permettant de signer des documents de manière authentique et sécurisée. Contrairement au papier, elle offre l'avantage de l'immédiateté et de la sécurité. Par exemple, une signature électronique simple peut être un simple scan de votre signature manuscrite apposé sur un document numérique. Cependant, des niveaux supérieurs de signatures, tels que les signatures électroniques avancées ou qualifiées, utilisent des certificats numériques délivrés par des pr

In [114]:
final_articles[0]

'```markdown\n# Comment faire une signature électronique : Guide complet\n\nBienvenue dans notre guide complet sur la réalisation de signatures électroniques. Que vous soyez un particulier cherchant à signer des documents à distance ou une entreprise souhaitant accélérer ses processus administratifs, ce guide vous fournira toutes les informations nécessaires. Nous répondrons à vos questions sur la définition de la signature électronique, les outils à utiliser, les étapes à suivre et la validité légale de ce procédé.\n\n## Qu\'est-ce qu\'une signature électronique ?\n\nPour comprendre pleinement une signature électronique, il est important de commencer par sa définition et ses avantages par rapport à une signature manuscrite.\n\n### Définition et avantages\n\nUne signature électronique se définit comme "une suite de données permettant d\'identifier le signataire d\'un document électronique et de garantir l\'intégrité de ce document". Contrairement à une signature manuscrite, qui est app

In [145]:
def raw_md_to_list_hn(markdown_text):

    """
    Extraire structure HN initialement en markdown pour la transformer en liste de liste [niveau, texte]

    Args:
        markdown_text: texte en input

    Returns: Structure HN en liste

    """
    # Utiliser une expression régulière pour identifier les titres Hn
    hn_pattern = re.compile(r'^(#+)\s*(.+)')

    # Initialiser une liste pour stocker la structure Hn
    structure = []

    # Parcourir chaque ligne du texte markdown
    for line in markdown_text.splitlines():
        match = hn_pattern.match(line)
        if match:
            # Obtenir le niveau de l'en-tête (nombre de '#')
            level = len(match.group(1))
            # Obtenir le titre de l'en-tête
            title = match.group(2).strip()
            # Ajouter à la structure en utilisant le format ["hN", "titre"]
            structure.append([f"h{level}", title])

    return structure

In [146]:
raw_md_to_list_hn(final_articles[0])

[['h1', 'Comment faire une signature électronique : Guide complet'],
 ['h2', "Qu'est-ce qu'une signature électronique ?"],
 ['h3', 'Définition et avantages'],
 ['h3', 'Différence avec la signature manuscrite'],
 ['h2', 'Comment fonctionne une signature électronique ?'],
 ['h3', 'Les principes technologiques'],
 ['h3', 'Les étapes de vérification'],
 ['h2', 'Étapes pour créer une signature électronique'],
 ['h3', 'Choisir le bon outil ou logiciel'],
 ['h3', 'Configurer votre compte'],
 ['h3', 'Importer des documents'],
 ['h3', 'Signer électroniquement'],
 ['h3', 'Sauvegarder et vérifier la signature'],
 ['h2', 'Les meilleurs outils pour créer une signature électronique'],
 ['h3', 'Comparaison des outils populaires'],
 ['h3', 'Caractéristiques clés à rechercher'],
 ['h2', 'Légalité et sécurité des signatures électroniques'],
 ['h3', 'Réglementations par pays'],
 ['h3', 'Mesures de sécurité'],
 ['h2', 'FAQ : Questions fréquentes'],
 ['h3', 'Comment signer un document PDF ?'],
 ['h3', 'Est

#### Proposition titre

In [75]:
concurrents_titre =  [[('titre', parsed_res["concurrents_data"][i]["title"]),
                      ('description', parsed_res["concurrents_data"][i]["descr"])] for i in range(n_concurrents) ]
"\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_titre ])


'## Début exemple \n[(\'titre\', \'Comment créer une signature électronique en ligne | Acrobat Sign\'), (\'description\', \'Découvrez comment créer une signature électronique vérifiable pour vos documents avec Acrobat Sign. Testez l’application gratuitement et commencez à signer des documents de manière rapide et sécurisée.\')]\n## Fin exemple \n \n## Début exemple \n[(\'titre\', \'La signature électronique : un outil devenu incontournable - francenum.gouv.fr\'), (\'description\', "Le développement du télétravail et la nécessité de gagner en efficacité et en rapidité rendent la faculté de signer à distance désormais indispensable au bon fonctionnement de l\'entreprise. La signature électronique permet en effet de signer instantanément, où que l\'on soit, la quasi totalité des actes administratifs ou commerciaux. Suivez le guide !")]\n## Fin exemple \n \n## Début exemple \n[(\'titre\', \'Comment faire une signature électronique ? - YouTube\'), (\'description\', "Découvrez comment signer

In [80]:
target_titre = [('titre', page_title), 
                ('description', page_description)]

In [87]:
prompt_titre = """
Vous êtes l'assistant d'un SEO qui doit améliorer le contenu d'une page web d'une entreprise, afin qu'il soit mieux référencés pour les recherches
des internautes. Vous avez ci-dessous quelques exemples de titre et description de de pages qui sont mieux référencés. Vous avez également le 
titre et la description de la page web de l'entreprise, à améliorer. Vous devez vous inspirer des titres et description des pages mieux référencés 
pour proposer une amélioration du titre et de la description de la page de l'entreprise. Votre réponse doit être uniquement une liste de tuples 
Python, comme dans les exemples, sans aucun commentaire.

Voici les titres et descriptions des pages les mieux référencées : 
---
{exemples_titres}
---

Voici la structure Hn de la page à améliorer : 
---
{titre_initial}
---

Voici un exemple de réponse : 

[('titre', "Mon titre 1"), ("description", "Ma description")]
""".replace("{exemples_titres}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_titre ]))\
.replace("{titre_initial}", str(target_titre))

In [88]:
prompt_titre

'\nVous êtes l\'assistant d\'un SEO qui doit améliorer le contenu d\'une page web d\'une entreprise, afin qu\'il soit mieux référencés pour les recherches\ndes internautes. Vous avez ci-dessous quelques exemples de titre et description de de pages qui sont mieux référencés. Vous avez également le \ntitre et la description de la page web de l\'entreprise, à améliorer. Vous devez vous inspirer des titres et description des pages mieux référencés \npour proposer une amélioration du titre et de la description de la page de l\'entreprise. Votre réponse doit être uniquement une liste de tuples \nPython, comme dans les exemples, sans aucun commentaire.\n\nVoici les titres et descriptions des pages les mieux référencées : \n---\n## Début exemple \n[(\'titre\', \'Comment créer une signature électronique en ligne | Acrobat Sign\'), (\'description\', \'Découvrez comment créer une signature électronique vérifiable pour vos documents avec Acrobat Sign. Testez l’application gratuitement et commencez

In [85]:
%%time
res_titre_gpt = gpt_llm.invoke(prompt_titre)
res_titre_gpt

CPU times: user 48.9 ms, sys: 20.8 ms, total: 69.7 ms
Wall time: 10.1 s


"[('titre', 'Créer une signature électronique en ligne avec Docaposte | Signature électronique pour documents'), ('description', 'Découvrir avec Docaposte comment créer une signature électronique fiable et sécurisée. Facilitez vos processus administratifs et commerciaux avec notre système de signature en ligne, un outil rapide, efficace et devenu incontournable pour signer instantanément, où que vous soyez.')]"

In [109]:
%%time
#res_titre_mistral = mistral_llm.invoke(prompt_titre)
#res_titre_mistral

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


#### Proposition contenu 

In [56]:
concurrents_content = [parsed_res["concurrents_data"][i]["content"] for i
                                            in range(n_concurrents)]
len(concurrents_content)

int(np.mean([len(x) for x in concurrents_content]))

3052

In [58]:
selected_structure = res_struct_gpt.structure['proposition1']
selected_structure

[['h1', 'Introduction à la signature électronique'],
 ['h2', 'Fonctionnement de la signature électronique'],
 ['h2', 'Comment effectuer une signature électronique sur un document'],
 ['h2', 'Méthodes pour créer une signature électronique'],
 ['h2', 'Comment signer électroniquement un document PDF'],
 ['h1', 'Outils pour la signature électronique'],
 ['h2', 'Logiciels de signatures électroniques'],
 ['h2', 'Signature électronique gratuite'],
 ['h1', 'Questions fréquentes']]

In [78]:
concurrents_content = [parsed_res["concurrents_data"][i]["content"] for i
                                            in range(10)]
len_content = int(np.mean([len(x) for x in concurrents_content]))
prompt_content = """
Vous êtes l'assistant d'un consultant SEO qui doit proposer un article sur un sujet pour le site web de son entreprise.
Pour cela, vous disposez de plusieurs exemples d'articles et d'une structure Hn à respecter, parlant du sujet.
Ces exemples doivent vous servir pour proposer 3 versions d'article pour aider le consultant SEO. Chaque partie de la
structure Hn devrait être remplie avec un contenu adapté. Vous pouvez vous baser également sur des connaissances générales
pour remplir le contenu et pas seulement les exemples de contenu fournis.
La structure Hn proposée est sous format de liste de tuples avec le niveau du titre (h1, h2, etc) et le texte du titre.
La réponse doit comporter environ {len_content} caractères. L
Votre réponse doit être uniquement un dictionnaire contenant 3 éléments et correspondant aux 3 contenus générés.
Le format de la réponse doit être comme dans les exemples ci-dessous, sans aucun commentaire.
Vous avez en entrée : le sujet pour le contenu à rédiger, les exemples de contenu, la structure Hn à respecter.

Voici le sujet pour le contenu à rédiger :
---
{keyword}
---

Voici les exemples de contenus :
---
{exemples_contenus}
---

Voici la structure Hn à respecter :
---
{structure_hn}
---

Voici le format de la réponse à donner:
{
    "proposition1" : "Comment faire une signature electronique : mode d'emploi. \n Pour faire une signature, il faut 3 élements.",
    "proposition2" : "La signature electronique en détail. \n Il existe plusieurs moyen pour faire une signature.",
    "proposition3" : "Faire une signature electronique. \n La signature électronique devient la norme."
}
""".replace("{exemples_contenus}", "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_content ]))\
.replace("{structure_hn}", str(selected_structure))\
.replace("{keyword}", str(key_word))


In [59]:
def generate_prompt_content(subject, structure_hn, examples, len_content):
    prompt = f"""
    Vous êtes un assistant et vous aidez un consultant SEO pour rédiger des contenus d'articles. 
    Je vais vous fournir une structure Hn à respecter pour rédiger l'article. 
    Cette structure est sous forme d'une liste de sous-listes du type ["h1", "titre de la section"]. 
    Je vais également vous donner plusieurs exemples d'articles pour vous inspirer. 
    Vous devez générer trois propositions de contenu d'article en respectant la structure Hn fournie.

    ### Entrées :

    1. **Sujet de l'article à rédiger** :
       {subject}

    2. **Structure Hn à respecter** :
       {structure_hn}

    3. **Exemples d'articles** :
       {examples}

    ### Sortie attendue :

    Un dictionnaire contenant trois propositions de contenu d'article, chaque proposition respectant la structure Hn 
    fournie et mis sous format de liste. Ces listes doivent contenir des sous-listes ayant trois éléments qui sont le 
    niveau du titre (h1, h2, etc), le texte du titre et le texte du contenu généré. Le format de ces sous-listes est 
    le suivant : ["h1", "le titre", "contenu généré"].

    Exemple de sortie :
    {{
        "proposition1": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ],
        "proposition2": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ],
        "proposition3": [
            ["h1", "Titre principal", "Contenu généré pour le titre principal"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"],
            ["h3", "Sous-sous-titre", "Contenu généré pour le sous-sous-titre"],
            ["h2", "Sous-titre", "Contenu généré pour le sous-titre"]
        ]
    }}

    ### Instructions :

    1. Respecter strictement la structure Hn fournie pour chaque proposition.
    2. S'inspirer des exemples d'articles fournis pour le ton et le style.
    3. Générer un contenu original et informatif pour chaque section de chaque proposition.
    4. Chaque proposition de contenu doit avoir environ {len_content} caractères de texte.
    5. Utiliser des guillemets doubles pour tout texte dans le résultat.
    """
    return prompt

len_content = int(np.mean([len(x) for x in concurrents_content]))

prompt_content2 = generate_prompt_content(key_word, selected_structure, 
                                          "\n".join(['## Début exemple \n' + str(i) + '\n## Fin exemple \n ' for i in concurrents_content ]),
                                         len_content)
len(prompt_content2)

37038

In [62]:
%%time
res_content_gpt = chain_content.invoke({"query": prompt_content2})
res_content_gpt

CPU times: user 57 ms, sys: 52.3 ms, total: 109 ms
Wall time: 2min 10s


Content(content={'proposition1': [['h1', 'Introduction à la signature électronique', "La signature électronique est un outil moderne qui facilite les échanges professionnels. Sa capacité à signer instantanément n'importe quel document est devenue essentielle pour les entreprises de toutes tailles. Mais comment fonctionne-t-elle et comment peut-on la créer ?"], ['h2', 'Fonctionnement de la signature électronique', "Au coeur de son système, la signature électronique utilise une technologie de cryptographie pour garantir l'authenticité du signataire et l'intégrité du document. Le but principal est donc de s'assurer que le document n'a pas été modifié une fois signé et que l'auteur du document est bien celui qu'il prétend être."], ['h2', 'Comment effectuer une signature électronique sur un document', "Plusieurs étapes sont nécessaires pour créer une signature électronique. Après avoir reçu une invitation par email pour signer un document, l'utilisateur doit confirmer son consentement et in

In [69]:
a, b, c = res_content_gpt.content['proposition1'][0]
c

"La signature électronique est un outil moderne qui facilite les échanges professionnels. Sa capacité à signer instantanément n'importe quel document est devenue essentielle pour les entreprises de toutes tailles. Mais comment fonctionne-t-elle et comment peut-on la créer ?"

In [83]:
%%time
res_content_gpt = gpt_llm.invoke(prompt_content)
res_content_gpt

CPU times: user 47.7 ms, sys: 11.1 ms, total: 58.8 ms
Wall time: 1min 47s


'{\n    "proposition1" : {\n        "h1": "Comprendre la signature électronique",\n        "h2": ["La signature électronique est un mécanisme permettant de garantir l\'intégrité d\'un document électronique, d\'authentifier son auteur et de rapporter la preuve du consentement. C\'est un moyen de signer instantanément, où que l\'on soit, la quasi totalité des actes administratifs ou commerciaux.", \n                "Il existe plusieurs types de signatures électroniques : simple, avancée ou qualifiée, qui varient en termes de niveau de fiabilité et de sécurité.", \n                "La signature électronique offre de nombreux avantages : elle facilite l\'envoi et l\'échange des documents, accélère la procédure de signature et sécurise les données. Cependant, tous les types de signatures électroniques ne disposent pas du même degré de force probante."],\n\n        "h1": "Comment réaliser une signature électronique",\n        "h2": ["Pour faire une signature électronique, il faut d\'abord té

In [70]:
%%time
res_content_mistral = mistral_llm.invoke(prompt_content)
res_content_mistral

CPU times: user 12.1 ms, sys: 10.7 ms, total: 22.8 ms
Wall time: 49.7 s


'\nVoici la réponse :\n\n{\n    "proposition1" : "Comment faire une signature électronique : mode d\'emploi. <br> Pour faire une signature électronique, il faut trois éléments : un document à signer, un outil de signature électronique et un signataire.<br><br> **Étape 1 : Téléchargez vos documents**<br> Téléchargez vos documents au format PDF, Word, Excel ou tout autre format à partir de votre ordinateur, de votre smartphone ou d\'un site de partage de fichiers.<br><br> **Étape 2 : Ajoutez les signataires**<br> Indiquez le nom et l\'adresse e-mail de vos signataires et précisez l\'ordre dans lequel ils doivent recevoir le document à signer.<br><br> **Étape 3 : Disposez des champs et envoyez**<br> Placez les champs de signature électronique par glisser-déposer à l\'endroit du fichier où vous avez besoin d\'une signature, d\'un paraphe ou de dates.<br><br> Cliquez ensuite sur Envoyer. DocuSign envoie par un mail un lien à chaque destinataire sur lequel ils doivent cliquer pour accéder au

In [81]:
%%time
res_content_gpt2 = gpt_llm.invoke(prompt_content2)
res_content_gpt2

CPU times: user 48.8 ms, sys: 70.7 ms, total: 119 ms
Wall time: 2min 24s


'{\n    "proposition1": [\n        (\'h1\', \'Comprendre la signature électronique\', \n         "Des contrats d\'entreprise aux formulaires en ligne, la signature électronique est de plus en plus couramment utilisée pour confirmer son accord. Cette méthode numérique, à la fois pratique et valable juridiquement, offre une grande flexibilité."),\n        (\'h2\', "Qu\'est-ce qu\'une signature électronique ?",\n         "Il s\'agit de l\'équivalent numérique de votre signature manuscrite. Elle est utilisée pour approuver des informations ou des documents dans un format électronique. Cela peut être aussi simple qu\'un nom tapé, une image numérisée ou une marque unique créée via un outil numérique."),\n        (\'h2\', \'Les différents types de signature électronique\',\n         "Il existe plusieurs types, allant de la signature simple à la signature qualifiée. Le choix dépend de votre besoin en matière de sécurité et de validation. Le niveau de sécurité et d\'authentification varie en fo

In [85]:
def parse_text_to_dict(text: str) -> dict:
    s = text.find('{')
    e = text.rfind('}')
    result_dict = {}
    try:
        # Nettoyer le texte et transformer les fausses échappements
        clean_text = text[s:e + 1].replace("\\'", "'")
        # Utiliser ast.literal_eval pour évaluer le texte en un dictionnaire Python
        result_dict = ast.literal_eval(clean_text)
    except:
        pass
    return result_dict

In [86]:
parse_text_to_dict(res_content_gpt2)

{'proposition1': [('h1',
   'Comprendre la signature électronique',
   "Des contrats d'entreprise aux formulaires en ligne, la signature électronique est de plus en plus couramment utilisée pour confirmer son accord. Cette méthode numérique, à la fois pratique et valable juridiquement, offre une grande flexibilité."),
  ('h2',
   "Qu'est-ce qu'une signature électronique ?",
   "Il s'agit de l'équivalent numérique de votre signature manuscrite. Elle est utilisée pour approuver des informations ou des documents dans un format électronique. Cela peut être aussi simple qu'un nom tapé, une image numérisée ou une marque unique créée via un outil numérique."),
  ('h2',
   'Les différents types de signature électronique',
   "Il existe plusieurs types, allant de la signature simple à la signature qualifiée. Le choix dépend de votre besoin en matière de sécurité et de validation. Le niveau de sécurité et d'authentification varie en fonction du type."),
  ('h2',
   'Avantages et inconvénients de

In [54]:
%%time
#res_content_mistral2 = mistral_llm.invoke(prompt_content2)
#res_content_mistral2

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 28.1 µs


#### Proposition résumé 

In [45]:
import requests
from bs4 import BeautifulSoup, Tag
from nltk.tokenize import word_tokenize

def get_page_info(url):
    # Récupérer la page web
    response = requests.get(url)
    response.raise_for_status()  # Vérifier si la requête a réussi

    # Parser la page avec BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Récupérer le titre de la page
    title = soup.title.string if soup.title else 'No title found'

    # Récupérer la description
    description_tag = soup.find('meta', attrs={'name': 'description'})
    description = description_tag['content'] if description_tag else 'No description found'

    # Récupérer tout le contenu principal après le H1
    h1_tag = soup.find('h1')
    content = ""
    if h1_tag:
        # Parcourir les éléments suivants après le H1 jusqu'à la fin du document
        next_sibling = h1_tag.find_next_sibling()
        while next_sibling and isinstance(next_sibling, Tag):
            content += next_sibling.get_text(separator=" ", strip=True) + " "
            next_sibling = next_sibling.find_next_sibling()

    # Compter le nombre de mots dans le contenu
    words = word_tokenize(content)
    word_count = len(words)

    return {
        'title': title,
        'description': description,
        'word_count': word_count,
        'content': content.strip()
    }

# Exemple d'utilisation
url = "https://www.docaposte.com/blog/article/comment-creer-signature-electronique-document"
page_info = get_page_info(url)

print("Title:", page_info['title'])
print("Description:", page_info['description'])
print("Word Count:", page_info['word_count'])
print("Content:", page_info['content'])


Title: Comment faire une signature électronique sur un document ?
Description: La création d'une signature électronique se déroule généralement en trois étapes ! Découvrez ce qu'il faut savoir pour créer sa signature !
Word Count: 6
Content: Le 29.08.2024 Article | 5 min


In [46]:
import requests
from bs4 import BeautifulSoup

def extract_hn_structure_and_content(url):
    # Récupérer le contenu de la page web
    response = requests.get(url)
    response.raise_for_status()

    # Parser la page avec BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialiser la liste pour stocker la structure Hn avec le contenu
    hn_structure = []

    # Trouver toutes les balises Hn dans la page
    headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

    # Parcourir chaque balise Hn et extraire le contenu qui suit jusqu'à la prochaine balise Hn
    for i, header in enumerate(headers):
        # Déterminer le niveau de la balise (h1, h2, etc.)
        level = header.name
        
        # Le texte du titre
        title = header.get_text(strip=True)
        
        # Trouver tout le contenu jusqu'à la prochaine balise Hn de même ou de plus haut niveau
        content = []
        next_node = header.find_next_sibling()

        while next_node and (not next_node.name or not next_node.name.startswith('h') or int(next_node.name[1]) > int(level[1])):
            content.append(next_node.get_text(strip=True))
            next_node = next_node.find_next_sibling()

        # Joindre le contenu en une seule chaîne de texte
        content_text = ' '.join(content)
        
        # Ajouter à la structure
        hn_structure.append((level, title, content_text))

    return hn_structure

# Exemple d'utilisation
url = "https://www.docaposte.com/blog/article/comment-creer-signature-electronique-document"
hn_structure = extract_hn_structure_and_content(url)

# Affichage de la structure Hn avec le contenu
for level, title, content in hn_structure:
    print(f"{level.upper()}: {title}")
    print(content)
    print("-" * 50)


H1: Comment faire une signature électronique sur un document ?
Le 29.08.2024Article|5 min
--------------------------------------------------
H2: L'essentiel à retenir sur la création d'une signature électronique sur un document
Lasignature électronique remplace la signature manuscritedans la majorité des processus, facilitant la gestion des échanges avec clients, fournisseurs et collaborateurs. Elle garantit l’identité du signataire, son accord sur le contenu, et l’intégrité du document signé. Lacréation d’une signature électroniquese déroule généralement en trois étapes : Obtention du certificat de signatureauprès d’une autorité de certification.Choix d’un outil de signature électroniqueadapté.Apposition de la signature sur le document, en garantissant que le document est verrouillé et lié à l’identité du signataire. Les documents couramment signés électroniquement incluent contrats, bons de livraison, mandats, et accords de confidentialité. Il existetrois niveaux de signaturedéfinis 

In [49]:
def get_hn_structure_and_content(url):
    response = requests.get(url)
    response.raise_for_status()  # Vérifie si la requête a réussi

    soup = BeautifulSoup(response.content, 'html.parser')
    # Extraction du titre de la page
    page_title = soup.title.string.strip() if soup.title else ''

    # Extraction de la description de la page
    meta_description = soup.find('meta', attrs={'name': 'description'})
    page_description = meta_description['content'].strip() if meta_description else ''

    # Extraction de la structure Hn et du contenu associé
    hn_structure = []
    current_section = None

    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        level = tag.name
        
        # Trouver le contenu de la section
        content = []
        next_node = tag.find_next_sibling()
        while next_node and (
                not next_node.name or not next_node.name.startswith('h') or int(next_node.name[1]) > int(level[1])):
            content.append(next_node.get_text(strip=True))
            next_node = next_node.find_next_sibling()

        # Joindre le contenu en une seule chaîne de texte
        content_text = ' '.join(content)
        current_section = (tag.name.upper(), tag.get_text(strip=True), content_text)
        hn_structure.append(current_section)

    all_content = ""
    for level, title, content in hn_structure:
        all_content += f"{level.upper()}: {title} \n\n {content}"

    return {"page_title": page_title,
            "page_description": page_description,
            "hn_structure": hn_structure,
            "content": all_content
            }

url = "https://www.docaposte.com/blog/article/comment-creer-signature-electronique-document"
get_hn_structure_and_content(url)

{'page_title': 'Comment faire une signature électronique sur un document ?',
 'page_description': "La création d'une signature électronique se déroule généralement en trois étapes ! Découvrez ce qu'il faut savoir pour créer sa signature !",
 'hn_structure': [('H1',
   'Comment faire une signature électronique sur un document ?',
   'Le 29.08.2024Article|5 min'),
  ('H2',
   "L'essentiel à retenir sur la création d'une signature électronique sur un document",
   'Lasignature électronique remplace la signature manuscritedans la majorité des processus, facilitant la gestion des échanges avec clients, fournisseurs et collaborateurs. Elle garantit l’identité du signataire, son accord sur le contenu, et l’intégrité du document signé. Lacréation d’une signature électroniquese déroule généralement en trois étapes : Obtention du certificat de signatureauprès d’une autorité de certification.Choix d’un outil de signature électroniqueadapté.Apposition de la signature sur le document, en garantissa

In [53]:
len({"dfz" : 1, "df" : "edf"})

2

In [149]:
import yaml
from yaml.loader import SafeLoader
with open("../AgentSEO/app/utils/banned_expression.yml") as f:
    banned = yaml.load(f, Loader=SafeLoader)

In [152]:
[s.strip() for s in banned["expressions"]]

['Naviguer',
 'Témoignage',
 'Testament',
 'Dans un contexte',
 'Peut conduire à',
 'Est la clé pour',
 'Plongez dans',
 'Intrigant',
 'Holistique',
 'Permettez-moi',
 'Avis à',
 'Il est important de noter',
 'Essentiel',
 'crucial',
 'impératif',
 'vital',
 'Vibrant',
 'captivant',
 'Orchestrer',
 'Embarquer',
 'Suivez le guide',
 'En conclusion',
 'Labyrinthe',
 'Dans le monde de',
 'Dans un monde de',
 'outil incontournable',
 'suivez ces étapes',
 'Dans le paysage de',
 "A l'ère de",
 'Débloquer',
 'Rigoureux',
 'élégant',
 'unique',
 'En constante évolution',
 'Se débattre',
 'Faire écho',
 'Certainement',
 'en suivant ces étapes']